# Wa1 v1.0

Goals: 20-50% YTD  
Drawdown < 50%  
Strat name: Wa1 v1.0  
Stragey-Gen Description: Mean reversiona and averaging down on markets  
Markets to test: Mining, Tech, Biomass, Healthcare  
Test both minute and day data  
Periods to test: 2003-2005, 2008-2009, 2013-2015  
Entry Rules: Moving average < 10d moving average; If sotck goes down, average down  
Exit Rules: If stock is up_%, sell; If stock is down 20% stop doubling down  
Things to remember: The rule of never investing more than 2% of equity in a single trade  

### Initialize
Using 9 sector funds - will test with several variations of these markets

In [ ]:
def initialize(context):
    context.security_list = symbols('XLY',  # XLY Consumer Discrectionary SPDR Fund   
                           'XLF',  # XLF Financial SPDR Fund  
                           'XLK',  # XLK Technology SPDR Fund  
                           'XLE',  # XLE Energy SPDR Fund  
                           'XLV',  # XLV Health Care SPRD Fund  
                           'XLI',  # XLI Industrial SPDR Fund  
                           'XLP',  # XLP Consumer Staples SPDR Fund   
                           'XLB',  # XLB Materials SPDR Fund  
                           'XLU')  # XLU Utilities SPRD Fund

Next we will define the rebalance. The algorithm will rebalance every day (or the first trading day if it's a holiday). At 11 AM ET, which is 1 hour and 30 minutes after the market open.

In [ ]:
    schedule_function(rebalance, date_rules.week_start(days_offset=0), time_rules.market_open(hours = 1, minutes = 30))  

### Rebalance
The rebalance class. It will loop through each of the stocks and order the target percentage. We will start by defining 4 variables: the number of positions, the 10 day prices, the 10 day moving average, and the current price.

In [ ]:
    for sec in context.security_list:
        #Check positions of security
        positions = context.portfolio.positions(sec)
            
        #Price data
        10_day_mavg = data.history(sec, "price", 10, "1d")
        mavg1 = 10_day_mavg.mean()
        current_price = data.current(sec, 'price')

Now for the trading logic.

In [ ]:
if data.can_trade(sec):
                        
    if positions == 0 and current_price > mavg1: #If you have no shares
        order_target_percent(sec, .05)
        pricebuy = current_price
    elif positions > 0: #if you already own the stock
        if current_price > (context.portfolio.positions[sec].cost_basis * 1.02): #if the stock has 2% gain
            order_target_value(sec, 0) #sell all shares
        elif current_price < (context.portfolio.positions[sec].cost_basis * 1.02) and current_price > (context.portfolio.positions[sec].cost_basis * .8): #if the current price is between 2% gain and 20% loss
            amount = context.portfolio.positions[sec].amount
            order_target_percent(sec, (amount * 2))
        
    else:
        pass